In [130]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time

In [92]:
%store -r df

In [93]:
poke_link = {}

In [94]:
#setting up dictionary of pokemon and their smogon links
for i in range(len(df)):
    poke_link[df.iat[i,2]]=('https://www.smogon.com/dex/sm/pokemon/'+df.iat[i, 2]+'/')

In [43]:
all_poke_text = {}

In [45]:
#cutoff = dict(list(poke_link.items())[110:])

In [ ]:
#get smogon battle strategies for each pokemon
for i, (key, value) in enumerate(poke_link.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    try:
        driver.execute_script(f"location.href='{target_url}';")
    except:
        print("url syntax issue")
    time.sleep(5)
    
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""
    try:
        paragraphs = poke_text.find_all("p")


        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("No text available")

    all_poke_text[key] = text
    #print(key, all_poke_text[key])
            

In [47]:
smog_df = pd.DataFrame(all_poke_text.items())

In [48]:
smog_df.head()

,0,1
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,
4,Charmander,Charmander might look like a good Pokemon to u...


In [49]:
#rename columns
smog_df.columns=["Pokemon", "Smogon_Description"]

In [50]:
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,
4,Charmander,Charmander might look like a good Pokemon to u...


In [51]:
smog_df.to_csv('smog_df.csv')

In [52]:
%store smog_df

Stored 'smog_df' (DataFrame)


In [158]:
%store -r smog_df

In [159]:
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,"Venusaur is an excellent choice for sun teams,...","Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...


In [153]:
#get missing entries
missing_entries = list(smog_df[smog_df.eq("").any(axis=1)]["Pokemon"])

In [154]:
missing_ent_dict = {}

In [155]:
#reformatting smogon links to handle special case pokemon (ex: Galarian Farfetch'd)
for ind, elem in enumerate(missing_entries)):
    key = elem
    if " " in missing_entries[ind]:
        missing_entries[ind] = elem.replace(" ", "-")
    if "Mega" in missing_entries[ind] or "Alolan" in missing_entries[ind] or "Galarian" in missing_entries[ind]:
        if "Alolan" in missing_entries[ind]:
            missing_entries[ind] = elem[elem.index("-")+1:] + "-" + elem[:elem.index("-")-1]
        elif "Galarian" in missing_entries[ind]:
            missing_entries[ind] = elem[elem.index("-")+1:] + "-" + elem[:elem.index("-")-3]
        else:
            missing_entries[ind] = elem[elem.index("-")+1:] + "-" + elem[:elem.index("-")]
    if "." in missing_entries[ind]:
        missing_entries[ind] = elem.replace(".", "")
    if "'" in missing_entries[ind]:
        missing_entries[ind] = elem.replace("'", "")
    if "♀" in missing_entries[ind]:
        missing_entries[ind] = elem.replace("♀", "-f")
    if "♂" in missing_entries[ind]:
        missing_entries[ind] = elem.replace("♂", "-m")
    

    missing_ent_dict[key] = 'https://www.smogon.com/dex/sm/pokemon/' + missing_entries[ind] + "/"


In [ ]:
missing_entries

In [ ]:
for i, (key, value) in enumerate(missing_ent_dict.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""
    try:
        paragraphs = poke_text.find_all("p")

        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        try:
            print("trying SS pokedex...")
            new_value = value.replace("/sm/", "/ss/")
            target_url = new_value
            print(target_url, "\n")
            
            options= Options()
            options.add_argument("-headless")
            driver = webdriver.Firefox(options=options)
            driver.execute_script(f"location.href='{target_url}';")
            time.sleep(5)
    
            resp = driver.page_source
            soup = BeautifulSoup(resp) 
            driver.close()
            
            poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
            try:
                paragraphs = poke_text.find_all("p")

                for para in paragraphs:
                    text+=para.get_text() + "\n"
            except:
                print("No text available")
        except:
            print("No text available")
            

    smog_df.loc[smog_df['Pokemon'] == key, "Smogon_Description"] = text

In [151]:
#see smogon description of select pokemon
#smog_df.loc[smog_df['Pokemon'] == "Venusaur"]["Smogon_Description"]

2    Venusaur is an excellent choice for sun teams,...
Name: Smogon_Description, dtype: object

In [165]:
%store smog_df

Stored 'smog_df' (DataFrame)


In [166]:
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,"Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...


In [173]:
missing_entries = list(smog_df[smog_df.eq("").any(axis=1)]["Pokemon"] + " smogon")

In [201]:
#reformatting
for index, elem in enumerate(missing_entries):
    if "'" in elem:
        missing_entries[index] = elem.replace("'", "")

In [207]:
#cutoff = missing_entries[82:]

'Minior Meteor Form smogon'

In [ ]:
for i in range(len(missing_entries)):
    entry = missing_entries[i]
    poke_name = entry[:-7]
    target_url = "https://www.google.com/search?q="+entry

    print(poke_name)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    print("google")
    time.sleep(3)

    result = driver.find_element(By.TAG_NAME,'h3')
    result.click()
    print("smogon")
    time.sleep(5)
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""

    try:
        paragraphs = poke_text.find_all("p")

        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("lol")
            
    smog_df.loc[smog_df['Pokemon'] == poke_name, "Smogon_Description"] = text

In [218]:
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,"Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...


In [2]:
%store -r smog_df

In [3]:
missing_entries = list(smog_df[smog_df.eq("").any(axis=1)]["Pokemon"])

In [4]:
missing_entries

["Galarian Farfetch'd",
 'Ampharos',
 'Mega Ampharos',
 'Giratina Origin Forme',
 'Eelektrik',
 "Oricorio Pa'u Style",
 'Minior Meteor Form',
 'Barraskewda',
 'Eiscue Ice Face',
 'Eiscue Noice Face',
 'Morpeko Full Belly Mode',
 'Arctozolt']

In [232]:
#smog_df.loc[smog_df['Pokemon'] == "Landorus Incarnate Forme"]["Smogon_Description"]

752    Faster Foes: Landorus's poor bulk leaves it vu...
Name: Smogon_Description, dtype: object

In [5]:
#doing manually since so few, special cases
missing_ent_dict = {"Galarian Farfetch'd": "https://www.smogon.com/dex/ss/pokemon/farfetchd-galar/",
                    'Ampharos': "https://www.smogon.com/dex/xy/pokemon/ampharos/",
                    'Mega Ampharos' : "https://www.smogon.com/dex/xy/pokemon/ampharos/",
                    'Giratina Origin Forme': "https://www.smogon.com/dex/ss/pokemon/giratina-origin/",
                    'Eelektrik': "https://www.smogon.com/dex/ss/pokemon/eelektrik/",
                    "Oricorio Pa'u Style": "https://www.smogon.com/dex/sm/pokemon/oricorio-pau/",
                    'Minior Meteor Form': "https://www.smogon.com/dex/sm/pokemon/minior/",
                    'Barraskewda' : "https://www.smogon.com/dex/ss/pokemon/barraskewda/almost-any-ability/",
                    'Eiscue Ice Face': "https://www.smogon.com/dex/ss/pokemon/eiscue/",
                    'Eiscue Noice Face' : "https://www.smogon.com/dex/ss/pokemon/eiscue/",
                    'Morpeko Full Belly Mode' : "https://www.smogon.com/dex/ss/pokemon/morpeko/",
                    'Arctozolt': "https://www.smogon.com/dex/ss/pokemon/arctozolt/national-dex/"}
                    

In [9]:
for i, (key, value) in enumerate(missing_ent_dict.items()):

    target_url = value

    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""

    try:
        paragraphs = poke_text.find_all("p")

        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("lol")
            
    smog_df.loc[smog_df['Pokemon'] == key, "Smogon_Description"] = text

0 Galarian Farfetch'd https://www.smogon.com/dex/ss/pokemon/farfetchd-galar/
1 Ampharos https://www.smogon.com/dex/xy/pokemon/ampharos/
2 Mega Ampharos https://www.smogon.com/dex/xy/pokemon/ampharos/
3 Giratina Origin Forme https://www.smogon.com/dex/ss/pokemon/giratina-origin/
4 Eelektrik https://www.smogon.com/dex/ss/pokemon/eelektrik/
5 Oricorio Pa'u Style https://www.smogon.com/dex/sm/pokemon/oricorio-pau/
6 Minior Meteor Form https://www.smogon.com/dex/sm/pokemon/minior/
7 Barraskewda https://www.smogon.com/dex/ss/pokemon/barraskewda/almost-any-ability/
8 Eiscue Ice Face https://www.smogon.com/dex/ss/pokemon/eiscue/
9 Eiscue Noice Face https://www.smogon.com/dex/ss/pokemon/eiscue/
10 Morpeko Full Belly Mode https://www.smogon.com/dex/ss/pokemon/morpeko/
11 Arctozolt https://www.smogon.com/dex/ss/pokemon/arctozolt/national-dex/


In [12]:
missing_entries = list(smog_df[smog_df.eq("").any(axis=1)]["Pokemon"])

In [14]:
%store smog_df

Stored 'smog_df' (DataFrame)


In [15]:
smog_df.to_csv('smog_df.csv')

In [16]:
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,"Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...


In [59]:
%store -r smog_df
smog_df.head()

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,No movesets available. Perhaps you could help ...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,"Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...


In [81]:
#checking for all invalid/missing entries that are not empty string
invalid_entries = list(smog_df[smog_df['Smogon_Description'].str.contains("No movesets available")]["Pokemon"])

In [ ]:
invalid_entries

In [83]:
len(invalid_entries)

139

In [84]:
invalid_ent_dict = {}

In [85]:
#reformatting smogon links, changing pokedex to SS 
for ind, elem in enumerate(invalid_entries):
    key = elem
    if " " in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace(" ", "-")
    if "Alolan" in invalid_entries[ind] or "Galarian" in invalid_entries[ind]:
        if "Alolan" in invalid_entries[ind]:
            invalid_entries[ind] = elem[elem.index(" ")+1:] + "-" + elem[:elem.index(" ")-1]
        elif "Galarian" in invalid_entries[ind]:
            invalid_entries[ind] = elem[elem.index(" ")+1:] + "-" + elem[:elem.index(" ")-3]
    if "." in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace(".", "")
    if "'" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("'", "")
    if "♀" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("♀", "-f")
    if "♂" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("♂", "-m")

    invalid_ent_dict[key] = 'https://www.smogon.com/dex/ss/pokemon/' + invalid_entries[ind] + "/"

In [ ]:
for i, (key, value) in enumerate(invalid_ent_dict.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""
    try:
        paragraphs = poke_text.find_all("p")    
        
        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("No text available")

    smog_df.loc[smog_df['Pokemon'] == key, "Smogon_Description"] = text

In [87]:
invalid_entries = list(smog_df[smog_df['Smogon_Description'].str.contains("No movesets available")]["Pokemon"])

In [ ]:
invalid_entries

In [89]:
len(invalid_entries)

120

In [94]:
invalid_ent_dict = {}

In [95]:
#repeating, except trying XY pokedex
for ind, elem in enumerate(invalid_entries):
    key = elem
    if " " in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace(" ", "-")
    if "Alolan" in invalid_entries[ind] or "Galarian" in invalid_entries[ind]:
        if "Alolan" in invalid_entries[ind]:
            invalid_entries[ind] = elem[elem.index(" ")+1:] + "-" + elem[:elem.index(" ")-1]
        elif "Galarian" in invalid_entries[ind]:
            invalid_entries[ind] = elem[elem.index(" ")+1:] + "-" + elem[:elem.index(" ")-3]
    if "." in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace(".", "")
    if "'" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("'", "")
    if "♀" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("♀", "-f")
    if "♂" in invalid_entries[ind]:
        invalid_entries[ind] = elem.replace("♂", "-m")

    invalid_ent_dict[key] = 'https://www.smogon.com/dex/xy/pokemon/' + invalid_entries[ind] + "/"

In [ ]:
invalid_ent_dict

In [ ]:
for i, (key, value) in enumerate(invalid_ent_dict.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""
    try:
        paragraphs = poke_text.find_all("p")    
        
        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("No text available")

    smog_df.loc[smog_df['Pokemon'] == key, "Smogon_Description"] = text

In [119]:
#entries that have movesets that are not being filled in
exist = ["Pikachu", "Seadra", "Magmar", "Porygon", "Vulpix", "Croconaw", "Spinark", "Togetic",
        "Marill", "Aipom", "Murkrow", "Marshtomp", "Wingull", "Surskit", "Electabuzz", "Gulpin", "Swablu", "Shelgon",
        "Prinplup", "Drifloon", "Happiny", "Sewaddle", "Whirlipede", "Gothita", "Quilladin", "Frogadier", "Sliggoo"]

In [117]:
invalid_entries = list(smog_df[smog_df['Smogon_Description'].str.contains("No movesets available")]["Pokemon"])

In [ ]:
invalid_entries

In [ ]:
missing_entries = list(smog_df[smog_df.eq("").any(axis=1)]["Pokemon"])
missing_entries

In [124]:
all_missing_entries = invalid_entries + missing_entries

In [121]:
len(all_missing_entries)

104

In [125]:
#get pokemon entries from smogon that exist (some pokemon have no battle strategies on smogon regardless of pokedex)
res_all_miss = list(set(all_missing_entries).intersection(exist))

In [126]:
len(res_all_miss)

5

In [127]:
print(res_all_miss)

['Happiny', 'Gulpin', 'Partner Pikachu', 'Drifloon', 'Pikachu']


In [128]:
miss_entr = {'Happiny': "https://www.smogon.com/dex/bw/pokemon/happiny/",
             'Gulpin': "https://www.smogon.com/dex/bw/pokemon/gulpin/",
             'Drifloon': "https://www.smogon.com/dex/sv/pokemon/drifloon/",
             'Pikachu': "https://www.smogon.com/dex/sv/pokemon/pikachu/"}

In [ ]:
for i, (key, value) in enumerate(miss_entr.items()):
    target_url = value
    
    print(i, key, value)
    
    options= Options()
    options.add_argument("-headless")
    driver = webdriver.Firefox(options=options)
    driver.execute_script(f"location.href='{target_url}';")
    time.sleep(5)
    
    resp = driver.page_source
    soup = BeautifulSoup(resp) 
    driver.close()
    
    poke_text = soup.find("section", {"data-reactid": ".0.1.1.2.6"})
    text = ""
    try:
        paragraphs = poke_text.find_all("p")    
        
        for para in paragraphs:
            text+=para.get_text() + "\n"
    except:
        print("No text available")

    smog_df.loc[smog_df['Pokemon'] == key, "Smogon_Description"] = text

In [131]:
%store smog_df

Stored 'smog_df' (DataFrame)


In [133]:
#replace invalid/missing entries with NaN
smog_df.loc[smog_df['Smogon_Description'].str.contains('No movesets available'), 'Smogon_Description'] = np.nan
smog_df.replace('-', np.nan)

,Pokemon,Smogon_Description
0,Bulbasaur,Bulbasaur is a potent sweeper in the LC metaga...
1,Ivysaur,The combination of Growth and Chlorophyll enab...
2,Venusaur,"Venusaur is an excellent choice for sun teams,..."
3,Mega Venusaur,"Venusaur is an excellent choice for sun teams,..."
4,Charmander,Charmander might look like a good Pokemon to u...
...,...,...
1040,Glastrier,240 Speed EVs outspeeds Corviknight.\nThis is ...
1041,Spectrier,Spectrier is almost completely outclassed by C...
1042,Calyrex,Don't use Calyrex.\n
1043,Calyrex Ice Rider,Thanks to a sky-high Attack stat coupled with ...


In [136]:
%store smog_df

Stored 'smog_df' (DataFrame)
